In [1]:
import os
os.chdir('/content/drive/MyDrive/LinguaTransform')
import sys
sys.path.append(os.getcwd())
from transformer import Transformer
import torch
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
english_file = '/content/drive/MyDrive/LinguaTransform/datasets/english_sentences.txt'
hindi_file = '/content/drive/MyDrive/LinguaTransform/datasets/hindi_sentences.txt'

# Generated this by filtering Appendix code

START_TOKEN = '<START>'
PADDING_TOKEN = '<PADDING>'
END_TOKEN = '<END>'

hindi_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', '?', 'ˌ',
                    'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ॠ', 'ऌ', 'ॡ', 'ए', 'ऐ', 'ओ', 'औ',
                    'क', 'ख', 'ग', 'घ', 'ङ',
                    'च', 'छ', 'ज', 'झ', 'ञ',
                    'ट', 'ठ', 'ड', 'ढ', 'ण',
                    'त', 'थ', 'द', 'ध', 'न',
                    'प', 'फ', 'ब', 'भ', 'म',
                    'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह',
                    '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'ॄ', 'े', 'ै', 'ो', 'ौ', '्', 'ॐ', '।', '॥',
                    '०', '१', '२', '३', '४', '५', '६', '७', '८', '९', PADDING_TOKEN, END_TOKEN]


english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x',
                        'y', 'z',
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [6]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocabulary)}
hindi_to_index = {v:k for k,v in enumerate(hindi_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [7]:
with open(english_file, 'r') as file:
    english_sentences = file.readlines()
with open(hindi_file, 'r') as file:
    hindi_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 200000
english_sentences = english_sentences[:TOTAL_SENTENCES]
hindi_sentences = hindi_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]

In [8]:
english_sentences[:10]

['politicians do not have permission to do what needs to be done',
 'id like to tell you about one such child',
 'this percentage is even greater than the percentage in india',
 'what we really mean is that theyre bad at not paying attention',
 'the ending portion of these vedas is called upanishad',
 'the then governor of kashmir resisted transfer  but was finally reduced to subjection with the aid of british ',
 'in this lies the circumstances of people before you',
 'and who are we to say even that they are wrong',
 'global warming refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being',
 'you may want your child to go to a school that is not run by the lea  a nonmaintained special school or an independent school that can meet your child s needs ']

In [9]:
hindi_sentences[:10]

['राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .',
 'मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,',
 'यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।',
 'हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते',
 'इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।',
 'कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी .',
 'इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।',
 'और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं',
 'ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।',
 'हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . .']

In [10]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length hindi: {np.percentile([len(x) for x in hindi_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 266.0
97th percentile length English: 260.0


In [11]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(hindi_sentences)):
    hindi_sentence, english_sentence = hindi_sentences[index], english_sentences[index]
    if is_valid_length(hindi_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(hindi_sentence, hindi_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 127607
Number of valid sentences: 101838


In [12]:
hindi_sentences = [hindi_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [13]:
hindi_sentences[:3]

['राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .',
 'मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,',
 'यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।']

In [14]:
import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
hi_vocab_size = len(hindi_vocabulary)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          hi_vocab_size,
                          english_to_index,
                          hindi_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [15]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(71, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [16]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

In [17]:
dataset = TextDataset(english_sentences, hindi_sentences)

In [18]:
len(dataset)

101838

In [19]:
dataset[1]

('id like to tell you about one such child',
 'मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,')

In [20]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [21]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('politicians do not have permission to do what needs to be done', 'id like to tell you about one such child', 'this percentage is even greater than the percentage in india', 'what we really mean is that theyre bad at not paying attention', 'the ending portion of these vedas is called upanishad', 'the then governor of kashmir resisted transfer  but was finally reduced to subjection with the aid of british ', 'in this lies the circumstances of people before you', 'and who are we to say even that they are wrong', 'you may want your child to go to a school that is not run by the lea  a nonmaintained special school or an independent school that can meet your child s needs ', 'category religious text', 'this period summarily is pepped up with devotion', 'so there is some sort of justice', 'the first two were found unreliable and the prosecution case rested mainly on the evidence of the remaining five approvers ', 'and now at present the naturecure ayurvedic and modern treatments are taking

In [22]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [23]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hi_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, hi_sentence_length = len(eng_batch[idx]), len(hi_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      hi_chars_to_padding_mask = np.arange(hi_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, hi_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, hi_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, hi_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

Modify mask such that the padding tokens cannot look ahead.
In Encoder, tokens before it should be -1e9 while tokens after it should be -inf.


In [25]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, hi_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hi_batch)
        optim.zero_grad()
        hi_predictions = transformer(eng_batch,
                                     hi_batch,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(hi_batch, start_token=False, end_token=True)
        loss = criterian(
            hi_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Hindi Translation: {hi_batch[0]}")
            hi_sentence_predicted = torch.argmax(hi_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in hi_sentence_predicted:
              if idx == hindi_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_hindi[idx.item()]
            print(f"Hindi Prediction: {predicted_sentence}")


            transformer.eval()
            hi_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hi_sentence)
                predictions = transformer(eng_sentence,
                                          hi_sentence,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_hindi[next_token_index]
                hi_sentence = (hi_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break

            print(f"Evaluation translation (should we go to the mall?) : {hi_sentence}")
            print("-------------------------------------------")

Epoch 0
Iteration 0 : 2.600588798522949
English: politicians do not have permission to do what needs to be done
Hindi Translation: राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .
Hindi Prediction: इारा    ार ं से क् ्हांके  राका   का ैक  किीही   कीयसनेते तह ै  है ह
Evaluation translation (should we go to the mall?) : ('और से का का का का हैं हैं हैं हैं हैं हैं हैं हैं हैं हैं है।<END>',)
-------------------------------------------
Iteration 100 : 2.6861071586608887
English: hindi literature
Hindi Translation: हिंदी साहित्य
Hindi Prediction: यि  ् केर त र कककककककककतककमककककककककककककककसककककककककककककककककककककककककककककककककककककककककककककककककककककककककककककमकककककककयकमकककककककककककककककककककककककककककककककककककककककककककककतपकककककककककककककककककककककवककककककककवककककयकककक
Evaluation translation (should we go to the mall?) : ('यह से के के के के के हैं हैं हैं हैं हैं<END>',)
-------------------------------------------
Iteration 200 : 2.6525814533233643
English: in dimensions it is much smaller than i

## Inference

In [26]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  hi_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hi_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_hindi[next_token_index]
    hi_sentence = (hi_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]

In [27]:
translation = translate("what should we do when the day starts?")
print(translation)

जब वो हम करने के लिए कि हम कर रहे हैं<END>


In [28]:
translation = translate("how is this the truth?")
print(translation)

वो उनके साथ है?<END>


In [29]:
translation = translate("the world is a large place with different people")
print(translation)

इस प्रकार के लिए कि विश्व का समय के लिए जाने के लिए हैं।<END>


In [30]:
translation = translate("my name is ajay")
print(translation)

मैं एक महान में हैं,<END>


In [31]:
translation = translate("i cannot stand this smell")
print(translation)

मैं एक प्रकार की संख्या है<END>


In [32]:
translation = translate("noodles are the best")
print(translation)

नेपाल के बारे में हैं<END>


In [33]:
translation = translate("why care about this?")
print(translation)

जब हम बात करते हैं<END>


In [34]:
translation = translate("this is the best thing ever")
print(translation)

यह इस बात करने के लिए हैं।<END>


The translation : "This is very unusual"

In [35]:
translation = translate("i am here")
print(translation)

मैं यह है<END>


In [38]:
translation = translate("click this")
print(translation)

श्रेणी:हिन्दी<END>


In [39]:
translation = translate("where is the mall?")
print(translation)

जब हम सब है,<END>


In [40]:
translation = translate("what should we do?")
print(translation)

जो कि हम करते हैं ?<END>


In [41]:
translation = translate("today, what should we do")
print(translation)

कि हम करने के लिए कि हम कर दिया है<END>


In [42]:
translation = translate("why did they activate?")
print(translation)

जो कि वो क्या है?<END>


In [43]:
translation = translate("why did they do this?")
print(translation)

जब हम क्या है?<END>


That turned out well!

In [44]:
translation = translate("i am well.")
print(translation)

मैं यह किया है ?<END>


Translation: "I will give you something"

In [45]:
translation = translate("whats the word on the street?")
print(translation)

जो कि वो क्या है?<END>


Overall, this model definately learned something. And you can use other languages instead of this Hindi language and might see better luck